In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import os

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d tongpython/cat-and-dog
!unzip cat-and-dog.zip -d /content/cats_and_dogs


Streaming output truncated to the last 5000 lines.
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3704.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3705.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3706.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3707.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3708.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3709.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.371.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3710.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3711.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3712.jpg  
  inflating: /content/cats_and_dogs/training_set/training_set/cats/cat.3713.jpg  
  inflating: /content/cats_and_dogs/training_set

In [ ]:
print(os.listdir("/content/cats_and_dogs"))



['test_set', 'training_set']


In [ ]:
train_dir = "/content/cats_and_dogs/training_set/training_set"
test_dir = "/content/cats_and_dogs/test_set/test_set"

IMG_SIZE = 128
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)


Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification (cat/dog)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,304,769 (12.61 MB)

 Trainable params: 3,304,769 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 26s 79ms/step - accuracy: 0.5421 - loss: 0.7142 - val_accuracy: 0.6742 - val_loss: 0.5972
Epoch 2/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.6864 - loss: 0.5816 - val_accuracy: 0.7014 - val_loss: 0.5768
Epoch 3/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 17s 68ms/step - accuracy: 0.7365 - loss: 0.5212 - val_accuracy: 0.7667 - val_loss: 0.4926
Epoch 4/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.7838 - loss: 0.4645 - val_accuracy: 0.7884 - val_loss: 0.4539
Epoch 5/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 17s 66ms/step - accuracy: 0.8046 - loss: 0.4138 - val_accuracy: 0.7865 - val_loss: 0.4665
Epoch 6/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.8538 - loss: 0.3359 - val_accuracy: 0.7963 - val_loss: 0.4715
Epoch 7/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.8819 - loss: 0.2828 - val_accuracy: 0.7884 - val_loss: 0.5067
Epoch 8/10
251/251 ━━━━━━━━━━━━━━━━━━━━ 14s 56ms/step - accuracy: 0.9137 - loss: 0.2145 - 

In [ ]:
loss, acc = model.evaluate(test_data)
print("Test Accuracy:", acc)

64/64 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - accuracy: 0.8095 - loss: 0.6117
Test Accuracy: 0.8022738695144653


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Change filename to your uploaded image
img_path = "/content/cats_and_dogs/training_set/training_set/cats/cat.1000.jpg"   # or whatever name Colab shows

IMG_SIZE = 128  # same size used in training
img = image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)  # batch of 1

prediction = model.predict(img_array)[0][0]

if prediction < 0.5:
    print("Prediction: 🐱 Cat")
else:
    print("Prediction: 🐶 Dog")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Prediction: 🐱 Cat


In [ ]:
model.save("/content/cat_dog_model.h5")